In [207]:
import os
from time import sleep
import requests
import re
from bs4 import BeautifulSoup
from pymorphy2 import MorphAnalyzer
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")
morph = MorphAnalyzer()

In [ ]:
# possible errors: no word in the dict ('div', class_="v3-none-text"), word with spelling mistake (тето ('div', class_="v2-spelling-flash-outer"))

In [ ]:
transforms = {'nom': 'nomn',
              'gen': 'gent',
              'dat': 'datv',
              'acc': 'accs',
              'ins': 'ablt',
              'loc': 'loct',
              'masc': 'masc',
              'fem': 'femn',
              'neut': 'neut'}

# All functions definition

In [401]:
def find_synonym(normal_form, gender=None):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    if os.path.exists('synonyms.txt'):
        with open('synonyms.txt') as synonyms_cache:
            synonyms_cache = [x.strip().split() for x in synonyms_cache]
            synonyms_cache = {(form, gen):syn for form, gen, syn in synonyms_cache}
        if (normal_form, str(gender)) in synonyms_cache:
            #print('yes')
            return synonyms_cache[(normal_form, str(gender))]
    try:
        content = requests.get(f'https://kartaslov.ru/синонимы-к-слову/{normal_form}', headers=headers).content
    except ConnectionError:
        print('Sleeping')
        sleep(5)
        try:
            content = requests.get(f'https://kartaslov.ru/синонимы-к-слову/{normal_form}', headers=headers).content
        except ConnectionError:
            print('no')
            return None
    soup = BeautifulSoup(content)
    #print(normal_form)
    if soup.findAll('div', class_="v3-none-text"):
        return None
    synonyms_page = soup.findAll('ul')
    if len(synonyms_page) > 1:
        synonyms = sum([x.text.strip().split(', ') for x in synonyms_page[1].findAll('li')], [])
        synonyms = [x for x in synonyms if ' ' not in x and '\xa0' not in x]
    else:
        return None
    
    # ADD CHECK WITH W2V
    #print(synonyms)
    if not synonyms:
        return None
    if gender:
        try:
            synonym = list(filter(lambda x: gender in x.tag, predictor.predict(synonyms)))[0].word
        except IndexError:
            synonym = None
    else:
        synonym = synonyms[0]
    #print(synonym)
    if synonym is not None:
        with open('synonyms.txt', 'a') as synonyms_cache:
            synonyms_cache.write(f'{normal_form} {gender} {synonym}'+'\n')
    return synonym

In [397]:
def replace_nouns(source_sentence, num=1):
    target_sentence = source_sentence.copy()

    candidate_forms = list(filter(lambda x: x.pos == 'NOUN', predictor.predict(source_sentence)))[:num]
    for form in candidate_forms:
        case_num = re.search('case=(.+)\|.+number=(.+)', form.tag.lower())
        gender = re.search('Gender=(.+)\|', form.tag)
        if gender:
            gender = gender.group(1)
        else:
            continue
        tags = {transforms[case_num.group(1)], case_num.group(2)}
        
        synonym = find_synonym(form.normal_form, gender)
        if synonym is None:
            continue
        inflected_word = list(filter(lambda x : x.tag.POS in ['NOUN'], morph.parse(synonym)))
        if inflected_word:
            #print(inflected_word[0], tags, inflected_word[0].inflect(tags))
            inflected_word = inflected_word[0].inflect(tags)
            if inflected_word:
                inflected_word = inflected_word.word
            else:
                continue
        else:
            continue
        target_sentence[target_sentence.index(form.word)] = inflected_word
    return target_sentence

In [398]:
def replace_adjectives(source_sentence, num=1):
    target_sentence = source_sentence.copy()

    candidate_forms = list(filter(lambda x: x.pos == 'ADJ', predictor.predict(source_sentence)))[:num]
    for form in candidate_forms:
        case = re.search('case=(.{3})\|.+', form.tag.lower())
        num = re.search('number=(.{4})', form.tag.lower())
        #print(form)
        if not num:
            continue
        if num.group(1) == 'plur':
            gender = None
            if case:
                tags = {transforms[case.group(1)], num.group(1)}
            else:
                tags = {num.group(1)}
        else:
            gender = re.search('gender=(.{3,4})\|.+', form.tag.lower()).group(1)
            if not case:
                #print(12, num.group(1), case_num)
                tags = {num.group(1), transforms[gender]}
            else:
                tags = {transforms[case.group(1)], num.group(1), transforms[gender]}
        
        #print(tags)
        synonym = find_synonym(form.normal_form)
        if synonym is None:
            continue
        inflected_word = list(filter(lambda x : x.tag.POS in ['ADJF'], morph.parse(synonym)))
        if inflected_word:
            inflected_word = inflected_word[0].inflect(tags).word
        else:
            continue
        target_sentence[target_sentence.index(form.word)] = inflected_word
    return target_sentence

In [411]:
def replace_adverbs(source_sentence, num=1):
    target_sentence = source_sentence.copy()

    candidate_forms = list(filter(lambda x: x.pos == 'ADV', predictor.predict(source_sentence)))[:num]
    for form in candidate_forms:
        synonym = find_synonym(form.normal_form)
        if synonym is None:
            continue
        else:
            target_sentence[target_sentence.index(form.word)] = synonym
    return target_sentence

In [402]:
def modify_sentences(src, tgt):
    modified_src = []
    modified_tgt = []
    for num, (src_sentence, sentence) in enumerate(zip(src, tgt)):
        #print(sentence)
        modified_sentence = replace_nouns(sentence)
        if modified_sentence == sentence:
            modified_sentence = replace_adverbs(sentence)
            if modified_sentence == sentence:
                modified_sentence = replace_adjectives(sentence)
                if modified_sentence == sentence:
                    continue
        modified_src.append(src_sentence)
        modified_tgt.append(modified_sentence)
        if num % 50 == 0:
            print(f'{num}/{len(src)}')
    return modified_src, modified_tgt

In [412]:
def augment(trainsetpath):
    if not os.path.exists(f'{trainsetpath}_aug'):
        os.mkdir(f'{trainsetpath}_aug')
    with open(f'{trainsetpath}/src-train14r.txt') as srctrain, open(f'{trainsetpath}/tgt-train14r.txt') as tgttrain, open(f'{trainsetpath}_aug/src-train.txt', 'w') as aug_srctrain, open(f'{trainsetpath}_aug/tgt-train.txt', 'w') as aug_tgttrain:
            srctrain = [x.strip().split() for x in srctrain]
            tgttrain = [x.strip().split() for x in tgttrain]
            aug_src, aug_tgt = modify_sentences(srctrain, tgttrain)
            for src_line, tgt_line in zip(srctrain, tgttrain):
                aug_srctrain.write(' '.join(src_line)+'\n')
                aug_tgttrain.write(' '.join(tgt_line)+'\n')
            for src_line, tgt_line in zip(aug_src, aug_tgt):
                aug_srctrain.write(' '.join(src_line)+'\n')
                aug_tgttrain.write(' '.join(tgt_line)+'\n')


0/4525
50/4525
100/4525
200/4525
250/4525
300/4525
400/4525
450/4525
500/4525
650/4525
700/4525
750/4525
800/4525
850/4525
950/4525
1000/4525
1050/4525
1150/4525
1200/4525
1250/4525
1300/4525
1350/4525
1500/4525
1550/4525
1700/4525
1900/4525
1950/4525
2000/4525
2100/4525
2150/4525
2200/4525
2250/4525
2300/4525
2350/4525
2400/4525
2500/4525
2600/4525
2700/4525
2750/4525
2800/4525
2850/4525
2900/4525
2950/4525
3000/4525
3100/4525
3150/4525
3250/4525
3300/4525
3350/4525
3400/4525
3450/4525
3500/4525
3550/4525
3600/4525
3750/4525
3850/4525
3900/4525
3950/4525
4000/4525
4050/4525
4100/4525
4150/4525
4250/4525
4350/4525
4400/4525
4450/4525
4500/4525


# Check that the functions work

In [133]:
with open('low_resource_mt/datasets/evenki/tgt-train.txt') as evenki:
    evenki = [x.strip().split() for x in evenki.readlines()]
evenki[:10]

[['так', 'живем', 'сейчас'],
 ['костры', 'разжег', 'отец'],
 ['потом', 'большой'],
 ['говорит', 'если', 'олени', 'потеряются'],
 ['у', 'тебя', 'опыт', 'большой'],
 ['я', 'в', 'берлогу', 'зашла'],
 ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'],
 ['оттуда', 'мы', 'в', 'стадо', 'аргишили'],
 ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'],
 ['сказала', 'это', 'самое']]

In [135]:
for sent in evenki[:10]:
    trans_sent = replace_nouns(sent)
    print(trans_sent, sent)

['так', 'живем', 'сейчас'] ['так', 'живем', 'сейчас']
['крепи', 'разжег', 'отец'] ['костры', 'разжег', 'отец']
['потом', 'большой'] ['потом', 'большой']
['говорит', 'если', 'лоси', 'потеряются'] ['говорит', 'если', 'олени', 'потеряются']
['у', 'тебя', 'навык', 'большой'] ['у', 'тебя', 'опыт', 'большой']
['я', 'в', 'нору', 'зашла'] ['я', 'в', 'берлогу', 'зашла']
['там', 'один', 'или', 'два', 'годка', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один']
['оттуда', 'мы', 'в', 'поголовье', 'аргишили'] ['оттуда', 'мы', 'в', 'стадо', 'аргишили']
['дед', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья']
['сказала', 'это', 'самое'] ['сказала', 'это', 'самое']


In [165]:
for sent in evenki[:10]:
    trans_sent = replace_adjectives(sent)
    print(trans_sent, sent)

['так', 'живем', 'сейчас'] ['так', 'живем', 'сейчас']
['костры', 'разжег', 'отец'] ['костры', 'разжег', 'отец']
nom sing
['потом', 'больший'] ['потом', 'большой']
['говорит', 'если', 'олени', 'потеряются'] ['говорит', 'если', 'олени', 'потеряются']
nom sing
['у', 'тебя', 'опыт', 'больший'] ['у', 'тебя', 'опыт', 'большой']
['я', 'в', 'берлогу', 'зашла'] ['я', 'в', 'берлогу', 'зашла']
nom sing
['там', 'какой-то', 'или', 'два', 'года', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один']
['оттуда', 'мы', 'в', 'стадо', 'аргишили'] ['оттуда', 'мы', 'в', 'стадо', 'аргишили']
['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья']
acc sing
['сказала', 'это', 'само'] ['сказала', 'это', 'самое']


In [178]:
for sent in evenki[:10]:
    trans_sent = replace_adverbs(sent)
    print(trans_sent, sent, trans_sent==sent)

['примерно', 'живем', 'сейчас'] ['так', 'живем', 'сейчас'] False
['костры', 'разжег', 'отец'] ['костры', 'разжег', 'отец'] True
['следом', 'большой'] ['потом', 'большой'] False
['говорит', 'если', 'олени', 'потеряются'] ['говорит', 'если', 'олени', 'потеряются'] True
['у', 'тебя', 'опыт', 'большой'] ['у', 'тебя', 'опыт', 'большой'] True
['я', 'в', 'берлогу', 'зашла'] ['я', 'в', 'берлогу', 'зашла'] True
['а там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'] ['там', 'один', 'или', 'два', 'года', 'все', 'время', 'один'] False
['оттудова', 'мы', 'в', 'стадо', 'аргишили'] ['оттуда', 'мы', 'в', 'стадо', 'аргишили'] False
['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] ['дедушка', 'это', 'кто', 'ходит', 'ломая', 'коряги', 'деревья'] True
['сказала', 'это', 'самое'] ['сказала', 'это', 'самое'] True
